In [1]:
import pandas as pd
import tweepy       # python wrapper for twitter api
import json
import time
import requests
# step 0, get your own twitter credentials!

In [2]:
# this is my own personal twitter api information
# if you could be so kind as to sign up yourself on both twitter and mashape that'd be great :)
# It's FREEEEEEE
api_key = 'NAw1wegVEZueRDqyA27c0pjjT'
api_secret = '0IbLwj3CPnsMs6xDPrRtjLDPvLAuYT1US6bgM5pvSlRemIFwrl'
access_token = '3278767399-BzpJJJxsA64HQqxPHRrjIaoNaPrz0mxfsOnl2Qu'
access_secret = 'Q2B7TbCNkZUTEsXfmFtpx9GlnMk3SimvW7WKmJC2waV0o'

In [3]:
# Documentation is your friend! http://docs.tweepy.org/en/v3.1.0/
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth) # returns a tweepy authorization handler object

In [4]:
# 1. Use Twitter API to stream and store tweets (300 per candidate)
# 37.781157,-122.39872 is the lat,lng for SF
hillary_tweets = api.search(q='#ImWithHer', count=100, geocode="37.781157,-122.398720,10mi") + api.search(q='#hillaryclinton', count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#hillary2016', count=100, geocode="37.781157,-122.398720,50mi")
the_donald_tweets = api.search(q='#donaldtrump', count=100, geocode="37.781157,-122.398720,10mi") + api.search(q='#trump2016', count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#trump', count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#makeamericagreatagain', count=100, geocode="37.781157,-122.398720,50mi")

In [5]:
the_donald_tweets[0].text

u"#stopandfrisk was deemed unconstitutional. If #DonaldTrump doesn't abide by the #constitution it's yet another reason he's unfit to serve."

In [6]:
hillary_tweets[0].text

u'Trump "sells" his fiction by constantly repeating his point. Supporters are mesmerized by repetition\n#NeverTrump\u2026 https://t.co/wjKU0pi2QH'

In [7]:
# wrappers come with built in python attributes and methods!
print dir(the_donald_tweets[0])

['__class__', '__delattr__', '__dict__', '__doc__', '__eq__', '__format__', '__getattribute__', '__getstate__', '__hash__', '__init__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_api', '_json', 'author', 'contributors', 'coordinates', 'created_at', 'destroy', 'entities', 'favorite', 'favorite_count', 'favorited', 'geo', 'id', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'metadata', 'parse', 'parse_list', 'place', 'retweet', 'retweet_count', 'retweeted', 'retweets', 'source', 'source_url', 'text', 'truncated', 'user']


In [8]:
print the_donald_tweets[0].created_at
print the_donald_tweets[0].text
print the_donald_tweets[0].favorite_count
print the_donald_tweets[0].favorited
print the_donald_tweets[0].lang
print the_donald_tweets[0].geo
print the_donald_tweets[0].retweet_count
print the_donald_tweets[0].retweeted
print the_donald_tweets[0].coordinates

2016-09-26 02:32:53
#stopandfrisk was deemed unconstitutional. If #DonaldTrump doesn't abide by the #constitution it's yet another reason he's unfit to serve.
0
False
en
None
0
False
None


In [9]:
# modify the tweets to make a nice dictionary
def parse_status_objects(tweet_list):
    modified_tweets = list()
    for x in tweet_list:
        modified_tweets.append( [
                    x.created_at,
                    x.text,
                    x.favorite_count,
                    x.favorited,
                    x.lang,
                    x.geo,
                    x.retweet_count,
                    x.retweeted,
                    x.coordinates
                ])
    return modified_tweets
        
hillary_tweets = parse_status_objects(hillary_tweets)
the_donald_tweets = parse_status_objects(the_donald_tweets)

In [10]:
print the_donald_tweets[0]
print hillary_tweets[0]

[datetime.datetime(2016, 9, 26, 2, 32, 53), u"#stopandfrisk was deemed unconstitutional. If #DonaldTrump doesn't abide by the #constitution it's yet another reason he's unfit to serve.", 0, False, u'en', None, 0, False, None]
[datetime.datetime(2016, 9, 26, 6, 0, 33), u'Trump "sells" his fiction by constantly repeating his point. Supporters are mesmerized by repetition\n#NeverTrump\u2026 https://t.co/wjKU0pi2QH', 0, False, u'en', None, 0, False, None]


In [11]:
# 2 Create a dataframe that combines all of the tweets from each candidate
# To do so, you will need to concat the data frames
# so each row is a tweet and your columns should be 
# date, text, favorite_count,favorited, language, geocode, retweet count, retweeted, coordinates

In [12]:
donald_df = pd.DataFrame(the_donald_tweets)
donald_df.rename(columns={0:'date', 1:'text', 2:'favorite_count',
                         3:'favorited', 4:'language', 5:'geocode',
                         6:'retweet_count', 7: 'retweeted',
                         8:'coordinates' }, inplace=True)


In [13]:
donald_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 9 columns):
date              310 non-null datetime64[ns]
text              310 non-null object
favorite_count    310 non-null int64
favorited         310 non-null bool
language          310 non-null object
geocode           0 non-null object
retweet_count     310 non-null int64
retweeted         310 non-null bool
coordinates       0 non-null object
dtypes: bool(2), datetime64[ns](1), int64(2), object(4)
memory usage: 17.6+ KB


In [14]:
hillary_df = pd.DataFrame(hillary_tweets)
hillary_df.rename(columns={0:'date', 1:'text', 2:'favorite_count',
                         3:'favorited', 4:'language', 5:'geocode',
                         6:'retweet_count', 7: 'retweeted',
                         8:'coordinates' }, inplace=True)

In [15]:
hillary_df

,date,text,favorite_count,favorited,language,geocode,retweet_count,retweeted,coordinates
0,2016-09-26 06:00:33,"Trump ""sells"" his fiction by constantly repeat...",0,False,en,None,0,False,None
1,2016-09-26 05:59:55,White House Press Secretary Josh Earnest on th...,0,False,en,None,0,False,None
2,2016-09-26 05:59:29,#BasketOfDeplorables #ImWithHer How to spot ...,0,False,en,None,0,False,None
3,2016-09-26 05:56:42,RT @Robrt_M_Goldste: #BasketOfDeplorables #Im...,0,False,en,None,1,False,None
4,2016-09-26 05:55:43,"Voted for Bernie, now #Imwithher. All of her ""...",0,False,en,None,0,False,None
5,2016-09-26 05:54:30,The Real Secret Behind Trump’s Terrorism Plan ...,0,False,en,None,0,False,None
6,2016-09-26 05:53:04,#BasketOfDeplorables #ImWithHer How to spot #...,0,False,en,None,1,False,None
7,2016-09-26 05:51:31,RT @Robrt_M_Goldste: #BasketOfDeplorables #Im...,0,False,en,None,1,False,None
8,2016-09-26 05:46:15,Anyone want to phone bank with me the evening ...,0,False,en,None,0,False,None
9,2016-09-26 05:45:52,#BasketOfDeplorables #ImWithHer How to Unders...,0,False,en,None,0,False,None


In [ ]:
# 3 Create a function to take in a string and output the textblob sentiment of that string

In [59]:
from textblob import TextBlob, Word

def detect_sentiment(text):
    return round(TextBlob(text).sentiment.polarity,3)

In [60]:
detect_sentiment('this is not good')

-0.35

In [61]:
detect_sentiment('this is awesome')

1.0

In [ ]:
# 4 Add a column to your data frame called 'sentiment'
# which holds the sentiment of that tweet (hint: use the function from #3)

In [62]:
donald_df['sentiment'] = donald_df.text.apply(detect_sentiment)

In [63]:
donald_df

,date,text,favorite_count,favorited,language,geocode,retweet_count,retweeted,coordinates,sentiment
0,2016-09-26 02:32:53,#stopandfrisk was deemed unconstitutional. If ...,0,False,en,None,0,False,None,0.000
1,2016-09-26 02:20:22,I believe #DonaldTrump won't win election desp...,0,False,en,None,0,False,None,0.000
2,2016-09-26 00:55:37,Doubt #DonaldTrump read this on #CascadeMallSh...,0,False,en,None,0,False,None,0.000
3,2016-09-26 00:04:58,A Different Kind of #DonaldTrump Story by @ton...,0,False,en,None,0,False,None,0.300
4,2016-09-26 00:04:55,RT @sonnylebythebay: Better vote for #DonaldTr...,0,False,en,None,1,False,None,0.108
5,2016-09-26 00:03:25,Better vote for #DonaldTrump or there'll be na...,0,False,en,None,1,False,None,0.108
6,2016-09-25 23:01:15,After @nytimes finds 31 #DonaldTrump lies in o...,0,False,en,None,0,False,None,0.000
7,2016-09-25 22:31:59,"#DonaldTrump text wants 200,000 people to give...",0,False,en,None,0,False,None,0.168
8,2016-09-25 22:04:29,RT @sfchronicle: Which #DonaldTrump will show ...,0,False,en,None,1,False,None,0.000
9,2016-09-25 22:03:03,Which #DonaldTrump will show up to Monday’s de...,4,False,en,None,1,False,None,0.000


In [64]:
hillary_df['sentiment'] = donald_df.text.apply(detect_sentiment)

In [65]:
hillary_df

,date,text,favorite_count,favorited,language,geocode,retweet_count,retweeted,coordinates,sentiment
0,2016-09-26 06:00:33,"Trump ""sells"" his fiction by constantly repeat...",0,False,en,None,0,False,None,0.000
1,2016-09-26 05:59:55,White House Press Secretary Josh Earnest on th...,0,False,en,None,0,False,None,0.000
2,2016-09-26 05:59:29,#BasketOfDeplorables #ImWithHer How to spot ...,0,False,en,None,0,False,None,0.000
3,2016-09-26 05:56:42,RT @Robrt_M_Goldste: #BasketOfDeplorables #Im...,0,False,en,None,1,False,None,0.300
4,2016-09-26 05:55:43,"Voted for Bernie, now #Imwithher. All of her ""...",0,False,en,None,0,False,None,0.108
5,2016-09-26 05:54:30,The Real Secret Behind Trump’s Terrorism Plan ...,0,False,en,None,0,False,None,0.108
6,2016-09-26 05:53:04,#BasketOfDeplorables #ImWithHer How to spot #...,0,False,en,None,1,False,None,0.000
7,2016-09-26 05:51:31,RT @Robrt_M_Goldste: #BasketOfDeplorables #Im...,0,False,en,None,1,False,None,0.168
8,2016-09-26 05:46:15,Anyone want to phone bank with me the evening ...,0,False,en,None,0,False,None,0.000
9,2016-09-26 05:45:52,#BasketOfDeplorables #ImWithHer How to Unders...,0,False,en,None,0,False,None,0.000


In [68]:
hillary_df.text[19]

u'RT @SF4Hillary: Thank you @jackie_emerson for stopping by and energizing our Sunday night phonebank at @SF4Hillary HQ! #ImWithHer #Stronger\u2026'

In [ ]:
# 5 create a word count column, which holds the number of words in the text column

In [77]:
def word_count_in_text(text):
    return len(text.split())

In [79]:
word_count_in_text('hello how you')

3

In [80]:
donald_df['word_count'] = donald_df.text.apply(word_count_in_text)

In [82]:
hillary_df['word_count'] = hillary_df.text.apply(word_count_in_text)

In [83]:
hillary_df

,date,text,favorite_count,favorited,language,geocode,retweet_count,retweeted,coordinates,sentiment,word_count
0,2016-09-26 06:00:33,"Trump ""sells"" his fiction by constantly repeat...",0,False,en,None,0,False,None,0.000,16
1,2016-09-26 05:59:55,White House Press Secretary Josh Earnest on th...,0,False,en,None,0,False,None,0.000,17
2,2016-09-26 05:59:29,#BasketOfDeplorables #ImWithHer How to spot ...,0,False,en,None,0,False,None,0.000,11
3,2016-09-26 05:56:42,RT @Robrt_M_Goldste: #BasketOfDeplorables #Im...,0,False,en,None,1,False,None,0.300,9
4,2016-09-26 05:55:43,"Voted for Bernie, now #Imwithher. All of her ""...",0,False,en,None,0,False,None,0.108,18
5,2016-09-26 05:54:30,The Real Secret Behind Trump’s Terrorism Plan ...,0,False,en,None,0,False,None,0.108,17
6,2016-09-26 05:53:04,#BasketOfDeplorables #ImWithHer How to spot #...,0,False,en,None,1,False,None,0.000,7
7,2016-09-26 05:51:31,RT @Robrt_M_Goldste: #BasketOfDeplorables #Im...,0,False,en,None,1,False,None,0.168,14
8,2016-09-26 05:46:15,Anyone want to phone bank with me the evening ...,0,False,en,None,0,False,None,0.000,27
9,2016-09-26 05:45:52,#BasketOfDeplorables #ImWithHer How to Unders...,0,False,en,None,0,False,None,0.000,9


In [ ]:
# use countvectorizer to create a document-term matrix and concatinate it to your main dataframe
#(hint you should now have thousands of columns)
# Don't add count vectorizer result to the original dataframe

In [ ]:
# 6 Use kmean to create clusters for the dataframe (choose an optimal k)
# Don't add clusters to the original dataframe

In [ ]:
# 7 Explore the clusters
# What is interesting? Surprising? Can you draw any inferences about each cluster?
# Write your answer to these questions in markdown

In [ ]:
# 8 Create a column called 'democrat' and assign it to the corresponding candidate name for all the records
# 1 for hilary and 0 for donald trump
# Make a boxplot of sentiment for each candidate by the "democrat" column

In [ ]:
# 9 Estimate classification models using knn and logistic regession.  Dont forget to use cross validation 
    # (do not use cross_val_score because we will want a confusion matrix).
# The cateogrical response here is predict democrat or republican
# Your features can EITHER be your countvectorizer document term matrix OR 
# data from the original dataframe (sentiment, word count, etc)

In [ ]:
# 10 Initialize a classification model (for both knn and logistic)
# This is separate from step 9 because I will eventually want a 
# confusion matrix and we can't do that via cros validation

In [ ]:
# 11 Split your data into training and testing tests and then
# train your classification model  (for both knn and logistic)

In [ ]:
# 12 Predict on your test features and targets (for both knn and logistic)

In [ ]:
# 13 output your accuracy (for both knn and logistic)

In [ ]:
# 14 Create a confusion matrix for your predictions (for both knn and logistic)
# What is the sensitivity and specificity?

In [ ]:
# 15 Who is the easiest candidate to predict? Why do you think? 
# Who is the hardest? Why do you think?
# Write your answers in markdown (for both knn and logistic)

In [ ]:
# 16 BONUS: make a pipeline with the first step being a feature union of countvectorizer
# and tfidfvectorizer and no other features and our second step is
# a ML model (your choice) predict our binary response


In [16]:
# 17: BONUS: grid search the model in step 16 to obtain the best features (up to you)

In [ ]:
# 18: SUPER BONUS: make a custom transformer that extracts sentiment and word count
# make a pipeline that combines countvec, tfidfvec and your custom transformer
# and try to get the possible accuracy for predicting whether or not the person is talking
# about democrats or republicans